# Анализ текстов

Интернет-магазин запускает новый сервис, чтобы пользователи могли редактировать и дополнять описания товаров, как в вики-сообществах. Магазину нужен инструмент, который будет определять токсичные комментарии и отправлять их на модерацию.

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. Заказчик предоставил набор данных с разметкой о токсичности правок.  Следует построить несколько разных моделей и рекомендовать ту, которая покажет наилучший результат. Задача - добиться метрики качества F1 не меньше 0.75.

Совместно с заказчиком определили список моделей для исследования:

- Логистическая регрессия
- Случайный лес
- CatBoost


# 1. Загрузка и исследование данных 


Загрузим необходимые для работы библиотеки.

In [1]:
#импорт библиотек 
import pandas as pd
import re


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords as nltk_stopwords 

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

In [2]:
#импортируем данные
data = pd.read_csv('toxic_comments.csv') 
data.head(10)


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [3]:
#удалим столбец-дубль индексов 'Unnamed: 0'
data = data.drop(columns = ['Unnamed: 0'],axis = 1)

Посмотрим на типы данных, количество строк и узнаем, есть ли пропуски

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропуски в данных отсутствуют.

# 1.2.  Проверка на дисбаланс классов

In [5]:
toxic_len = data[data['toxic'] == 1].shape[0]
toxic_non_len = data[data['toxic'] != 1].shape[0]

print('toxic: {:d} rows, {:.2%}'.format(toxic_len, toxic_len / data.shape[0]))
print('non_toxic: {:d} rows, {:.2%}'.format(toxic_non_len, toxic_non_len / data.shape[0]))

toxic: 16186 rows, 10.16%
non_toxic: 143106 rows, 89.84%


Соотношение классов -  1:9. Необходимо учесть это в параметрах моделей.

# 1.3 Нормализация и лемматизация текстов

In [6]:
# создадим функцию для очистки и лемматизации текста
nltk.download('wordnet')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()


def process_text(s):
    s = s.lower()
    words = [re.sub(r"[^a-zA-Z]", "", word) for word in word_tokenize(s)]
    words = [lemmatizer.lemmatize(word) for word in words if word != ""]

    return " ".join(words)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anastasiabatmanova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anastasiabatmanova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
#применим функцию для очистки  и лемматизации текста
data['text'] = data['text'].apply(process_text)

In [8]:
data.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,daww he match this background colour i m seemi...,0
2,hey man i m really not trying to edit war it s...,0
3,more i ca nt make any real suggestion on impro...,0
4,you sir are my hero any chance you remember wh...,0


Тексты готовы для превращения в признаки.

# 1.4. Подготовка массива признаков

Теперь надо обработать текст, чтобы превратить наборы слов в признаки. Используем технологию TF-IDF.
Также разобьем данные на выборки.

In [9]:

X_train, X_test, y_train, y_test = train_test_split(data['text'],data['toxic'], 
                                                    test_size=0.25, random_state=12345)

print("X_train.shape", X_train.shape)
print("y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape)
print("y_test.shape", y_test.shape)

X_train.shape (119469,)
y_train.shape (119469,)
X_test.shape (39823,)
y_test.shape (39823,)


In [10]:
#загрузим список стоп-слов
nltk.download('stopwords')
stop_words = set(nltk_stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anastasiabatmanova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#создадим счётчик, указав в нём стоп-слова
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
train_features = count_tf_idf.fit_transform(X_train.values.astype('U'))
test_features = count_tf_idf.transform(X_test.values.astype('U'))



# 2.  Обучение моделей

Обучим три разных модели:

- Логистическая регрессия
- Случайный лес
- CatBoost

Так как наша цель состоит в том, чтобы принципиально определить наилучшую модель, а массив данных получился большим, для сокращения времени подбор гиперпараматеров с помощью кросс-валидации проводить не будем. 

# 2.1 Логистическая регрессия


In [12]:
# инициализация логистической регрессии
model_lr = LogisticRegression(solver='lbfgs', 
                              max_iter=1000, 
                              class_weight='balanced',
                              random_state=12345, 
                              verbose=0)
                           

In [13]:
#обучаем модель логистической регрессии
model_lr.fit(train_features, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=12345)

In [15]:
#проверяем предсказание на валидационной выборке 
predict = model_lr.predict(test_features)
f1_lr = f1_score(y_test, predict)
print('LogistiсRegression F1 score -', f1_lr)

LogistiсRegression F1 score - 0.7471127900701565


# 2.2. Случайный лес


In [18]:
model_rf = RandomForestClassifier(
    n_estimators=1000, criterion='gini', max_depth=5, min_samples_split=2, min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, 
    random_state=12345, verbose=0, class_weight='balanced')
model_rf.fit(train_features, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=5, n_estimators=1000,
                       random_state=12345)

In [19]:
predict = model_rf.predict(test_features)
f1_rf = f1_score(y_test, predict)
print('RandomForestRegressor F1 score -', f1_rf)

RandomForestRegressor F1 score - 0.35177416966885455


# 2.3. CatBoost

In [ ]:
%%time
model_cat = CatBoostClassifier(
            n_estimators=1000, 
            class_weights=[1, 9],
            max_depth=4, 
            verbose=100)
model_cat.fit(train_features, y_train)

Learning rate set to 0.079419
0:	learn: 0.6606477	total: 832ms	remaining: 13m 50s
100:	learn: 0.4132445	total: 1m 5s	remaining: 9m 41s
200:	learn: 0.3599040	total: 2m 7s	remaining: 8m 27s
300:	learn: 0.3274321	total: 3m 9s	remaining: 7m 19s
400:	learn: 0.3052919	total: 4m 13s	remaining: 6m 18s
500:	learn: 0.2885927	total: 5m 22s	remaining: 5m 20s
600:	learn: 0.2744167	total: 6m 26s	remaining: 4m 16s
700:	learn: 0.2630890	total: 7m 23s	remaining: 3m 9s
800:	learn: 0.2533012	total: 8m 32s	remaining: 2m 7s
900:	learn: 0.2446372	total: 9m 32s	remaining: 1m 2s


In [ ]:
predict = model_cat.predict(test_features)
f1_cat = f1_score(y_test, predict)
print('CatBoostRegressor F1 score -', f1_cat)

# 3. Выводы



В ходе работы были сделаны следующие шаги:
- обнаружен дисбаланс классов в соотношении 1:9. Токсичные комментарии составляют примерно одну десятую часть набора текстов.
- проведена очистка и лемматизация текстов
- создан набор векторов признаков по технологии TF-IDF - относительная встречаемость слов в корпусе текстов
- обучены и проверены на тестовой выборке три модели: логистическая регрессия, случайный лес, CatBoost.



<div class="alert alert-block alert-success">
Наилучший результат показала модель CatBoost -  при помощи этой модели удалось добиться F1= 0.752. 
</div>